In [1]:
%load_ext autoreload
%autoreload 2

# Create Firebase Users

In [3]:
import names
import firebase_admin
from firebase_admin import auth, credentials

cred = credentials.Certificate("./../firebase-config-staging.json")
# app = firebase_admin.initialize_app(name="motivee", credential=cred)
app = firebase_admin.get_app(name="motivee")
client = auth.Client(app)

for i in range(250):
    display_name = names.get_full_name()  
    email = display_name.replace(" ", ".") + "@mail.com"
    client.create_user(email=email)


# Create Django Users

In [4]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
#
from login.models import User

page = client.list_users()
counter = 0
while page:
    for user in page.users:
        if not User.objects.filter(username=user.uid).exists():
            counter += 1 
            d_user = User(username=user.uid)
            d_user.save()
    page = page.get_next_page()

counter

250

# Create Tags

In [1]:
from conversations.models import Tag

TAGS = [
    'Budget',
    'Culture',
    'Engineering',
    'Leadership',
    'KPI',
    'Customers',
]

# for tag in TAGS:
#     Tag(name=tag.lower().strip()).save()

# Create Conversation

In [2]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from conversations.models import Conversation, Tag, Claim
from login.models import User


discussion_ids = ['36863', '10436', '14232', '16931', '18046', '25000', '48142']
to_pop = {
    '36863': ['36863.0', '36863.3'],
    '10436': ['10436.0', '10436.1'],
    '14232': ['14232.0'],
    '16931': ['16931.0', '16931.1'],
    '18046': ['18046.0', '18046.1'],
    '25000': ['25000.0', '25000.1'],
    '48142': ['48142.0'] 
}
questions_dict = {
    '36863': '36863.3',
    '10436': '10436.1',
    '14232': '14232.0',
    '16931': '16931.1',
    '18046': '18046.1',
    '25000': '25000.1',
    '48142': '48142.0' 
}


def get_tree_dict(discussion_id):
    with open('./data/tree_{}.json'.format(discussion_id), 'r') as file:
        tree_dict = json.load(file)
        
    question = tree_dict[questions_dict[discussion_id]]['text']    
    more_to_pop = []
    for idx, val in tree_dict.items():
        if val['parentId'] is None or val['parentId'] not in tree_dict:
            more_to_pop.append(idx)
            
    for idx, val in tree_dict.items():       
        if val['parentId'] in to_pop[discussion_id]:
            val['parentId'] = None
            
    for idx in to_pop[discussion_id]:
        if idx in tree_dict:
            tree_dict.pop(idx)
        
    for idx in more_to_pop:
        if idx in tree_dict:
            tree_dict.pop(idx)
             
    return tree_dict, question

type_dict = {
    0: 'ANS', -1: 'CON', 1: 'PRO'
}

sub_discussion_ids = [
#     '36863', 
    '14232', 
    '48142'
]

for idx in sub_discussion_ids:
    
    tree_dict, question = get_tree_dict(idx)
    conversation = Conversation(
        title=question[:120],
        question=question[:500],
        open_ended=(len(to_pop[idx]) == 1),
        author=random.choice(User.objects.all()), 
    )
    
    conversation.save()
    tags = random.sample(TAGS, k=5)
    conversation.tags.set([Tag.objects.get(name=tag.lower().strip()) for tag in tags])
    
    idx_dict = {}
    
    for claim_idx, claim_dict in tree_dict.items():
        
        
        print(idx, claim_idx)
        claim = Claim(
            text=claim_dict['text'][:500],
            author=random.choice(User.objects.all()),
            conversation=conversation,
            type=type_dict[claim_dict['relation']],     
        )
        
        
        try:
            claim.save()
            idx_dict[claim_idx] = claim.id
            claim_tags = random.sample(tags, k=random.randint(0, 3))
            claim.tags.set([Tag.objects.get(name=tag.lower().strip()) for tag in claim_tags])
        except Exception as e: 
            print(e)
            claim.type = 'ANS'
            claim.save()
            
            idx_dict[claim_idx] = claim.id
            claim_tags = random.sample(tags, k=random.randint(0, 3))
            claim.tags.set([Tag.objects.get(name=tag.lower().strip()) for tag in claim_tags])
            
#         except Exception as e:
#             print(e)
    
    for claim_idx, claim_dict in tree_dict.items():
        
#         try:
        
        print(idx, claim_idx)

        parent_id = claim_dict['parentId']
        if parent_id is None or parent_id not in idx_dict:
            continue

        claim = Claim.objects.get(id=idx_dict[claim_idx])
        parent_claim = Claim.objects.get(id=idx_dict[parent_id])
        claim.parent = parent_claim
        claim.save(update_fields=['parent'])
            
#         except Exception as e:
#             print(e)
        
    
            

14232 14232.3
14232 14232.4
14232 14232.5
14232 14232.6
14232 14232.7
14232 14232.8
14232 14232.10
14232 14232.11
14232 14232.12
14232 14232.13
14232 14232.14
14232 14232.16
14232 14232.17
14232 14232.18
14232 14232.19
14232 14232.20
14232 14232.21
14232 14232.22
14232 14232.26
14232 14232.27
14232 14232.28
14232 14232.29
14232 14232.30
14232 14232.35
14232 14232.36
14232 14232.37
14232 14232.38
14232 14232.39
14232 14232.41
14232 14232.42
14232 14232.43
14232 14232.45
14232 14232.46
14232 14232.47
14232 14232.48
14232 14232.49
14232 14232.53
14232 14232.58
14232 14232.59
14232 14232.590
14232 14232.1120
14232 14232.385
14232 14232.387
14232 14232.389
14232 14232.391
14232 14232.401
14232 14232.403
14232 14232.405
14232 14232.407
14232 14232.409
14232 14232.324
14232 14232.422
14232 14232.424
14232 14232.326
14232 14232.426
14232 14232.328
14232 14232.329
14232 14232.427
14232 14232.331
14232 14232.429
14232 14232.1123
14232 14232.431
14232 14232.333
14232 14232.433
14232 14232.335
142

14232 14232.847
14232 14232.849
14232 14232.851
14232 14232.853
14232 14232.855
14232 14232.857
14232 14232.859
14232 14232.861
14232 14232.863
14232 14232.865
14232 14232.867
14232 14232.869
14232 14232.871
14232 14232.873
14232 14232.875
14232 14232.879
14232 14232.884
14232 14232.886
14232 14232.888
14232 14232.890
14232 14232.892
14232 14232.894
14232 14232.896
14232 14232.898
14232 14232.904
14232 14232.906
14232 14232.908
14232 14232.910
14232 14232.912
14232 14232.914
14232 14232.916
14232 14232.918
14232 14232.920
14232 14232.697
14232 14232.683
14232 14232.52
14232 14232.729
14232 14232.173
14232 14232.33
14232 14232.1
14232 14232.15
14232 14232.775
14232 14232.843
14232 14232.31
14232 14232.735
14232 14232.61
14232 14232.827
14232 14232.54
14232 14232.40
14232 14232.9
14232 14232.32
14232 14232.739
14232 14232.183
14232 14232.231
14232 14232.165
14232 14232.610
14232 14232.56
14232 14232.24
14232 14232.25
14232 14232.2
14232 14232.55
14232 14232.60
14232 14232.133
14232 14232

In [33]:
claim.type

'ANS'

# Create Votes

In [8]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# django.setup()
from conversations.models import Conversation, Claim, Vote
from login.models import User

MAX_VOTES = 50

all_claims = Claim.objects.all()
all_users = User.objects.all()
all_conversations = Conversation.objects.all()

for conversation in all_conversations:
    conversation.votes = 0
    conversation.save(update_fields=['votes'])
    

for claim in all_claims:
    
    print(claim.id)
    
    up_votes = random.randint(0, MAX_VOTES)
    down_votes = random.randint(0, MAX_VOTES)
    
    voters = random.sample(list(all_users), k=(up_votes +  down_votes))
    up_voters = voters[:up_votes]
    down_voters = voters[up_votes:]
    
    claim.up_votes = len(up_voters)
    claim.down_votes = len(down_voters)
    claim.save(update_fields=['up_votes', 'down_votes'])
    
    conversation = claim.conversation
    
    conversation.votes += len(up_voters) + len(down_voters)
    conversation.save(update_fields=['votes'])
    
    for voter in up_voters:
        
        vote = Vote(
            user=voter,
            claim=claim,
            value=1, 
        )
        vote.save()
        
    for voter in down_voters:
        
        vote = Vote(
            user=voter,
            claim=claim,
            value=-1, 
        )
        vote.save()
        
    

4215
4214
4213
4212
4211
4210
4209
4208
4207
4206
4205
4204
4203
4202
4201
4200
4199
4198
4197
4196
4195
4194
4193
4192
4191
4190
4189
4188
4187
4186
4185
4184
4183
4182
4181
4180
4179
4178
4177
4176
4175
4174
4173
4172
4171
4170
4169
4168
4167
4166
4165
4164
4163
4162
4161
4160
4159
4158
4157
4156
4155
4154
4153
4152
4151
4150
4149
4148
4147
4146
4145
4144
4143
4142
4141
4140
4139
4138
4137
4136
4135
4134
4133
4132
4131
4130
4129
4128
4127
4126
4125
4124
4123
4122
4121
4120
4119
4118
4117
4116
4115
4114
4113
4112
4111
4110
4109
4108
4107
4106
4105
4104
4103
4102
4101
4100
4099
4098
4097
4096
4095
4094
4093
4092
4091
4090
4089
4088
4087
4086
4085
4084
4083
4082
4081
4080
4079
4078
4077
4076
4075
4074
4073
4072
4071
4070
4069
4068
4067
4066
4065
4064
4063
4062
4061
4060
4059
4058
4057
4056
4055
4054
4053
4052
4051
4050
4049
4048
4047
4046
4045
4044
4043
4042
4041
4040
4039
4038
4037
4036
4035
4034
4033
4032
4031
4030
4029
4028
4027
4026
4025
4024
4023
4022
4021
4020
4019
4018
4017
4016


# Update p-value

In [1]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from conversations.models import Claim

for i, claim in enumerate(Claim.objects.all()):
    print(i)
    claim.save()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# update Root Claim

In [3]:
from conversations.models import Conversation
from collections import deque


def get_bfs(root_claim):
    
    queue = deque([root_claim])
    while queue:
        
        claim = queue.popleft()
        print(claim.id)
        if claim != root_claim:
            claim.root_claim = root_claim
            claim.save(update_fields=['root_claim'])
        
        for child in claim.children.all():
            queue.append(child)
               

for conversation in Conversation.objects.all():
    if conversation.open_ended:
        root_claims = conversation.claims.filter(parent__isnull=True)
        
        for root_claim in root_claims.all():
            get_bfs(root_claim)
            
     

4191
4192
4190
4189
4186
4211
4197
4193
4188
4187
4213
4208
4206
4205
4209
4210
4180
4212
4204
4203
4181
4182
4179
4171
4215
4173
4172
4152
4150
4160
4164
4161
4007
4112
4201
4149
4110
4198
4200
4199
4109
4108
4107
4185
4100
4202
4116
4114
4103
4102
4101
4084
4091
4085
4082
4106
4105
4095
4094
4093
4090
4089
4088
4087
4086
4083
4096
4097
4071
4078
4077
4075
4072
4079
4076
4074
4073
4068
4066
4207
4067
4080
4040
4143
4070
4045
4038
4081
4069
4064
4039
4030
4036
4034
4033
4037
4044
4035
4029
4032
4031
4028
4115
4113
4098
4059
4052
4051
4050
4104
4024
4196
4195
4194
4184
4123
4057
4023
4169
4168
4167
4166
4155
4138
4129
4124
4120
4117
4058
4055
4053
4178
4170
4165
4157
4148
4140
4139
4133
4132
4128
4122
4119
4154
4144
4131
4130
4151
4141
4121
4127
4118
4153
4163
4162
4159
4156
4145
4136
4126
4125
4021
4158
4146
4047
4043
4042
4041
4060
4048
4046
4062
4020
4183
4177
4176
4175
4174
4147
4142
4054
4025
4049
4018
4137
4135
4056
4019
4065
4013
4134
4111
4061
4026
4017
4099
4011
4012
4010
4214


# Update top relation

In [4]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from conversations.models import Conversation
from collections import deque


def get_bfs(root_claim):
    
    queue = deque([root_claim])
    while queue:
        
        claim = queue.popleft()
        claim.save()
        
        for child in claim.children.all():
            queue.append(child)
               

for conversation in Conversation.objects.all():
    root_claims = conversation.claims.filter(parent__isnull=True)        
    for root_claim in root_claims.all():
        get_bfs(root_claim)

# Clean claim  types

In [5]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from conversations.models import Conversation
from collections import deque

for conversation in Conversation.objects.all():
    claims = conversation.claims      
    for claim in claims.all():
        if claim.parent is None and conversation.open_ended:
            claim.type = 'ANS'
        elif claim.type == 'ANS':
            claim.type = random.choice(['PRO', 'CON'])  
        claim.save()

# Impose Constraints
   

In [7]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from conversations.models import Conversation, Claim

for claim in Claim.objects.all():
    
    if claim.conversation.open_ended and claim.parent is None:
        print(1, claim)
        claim.type = 'ANS'
        claim.top_relation = 0
        claim.save()
        
    if claim.parent is not None and claim.type not in ('PRO', 'CON'):
        print(2, claim)
        claim.type = random.choice(['PRO', 'CON'])
        claim.save()
        
    if not claim.conversation.open_ended and claim.type not in ('PRO', 'CON'):
        print(3, claim)
        claim.type = random.choice(['PRO', 'CON'])
        claim.save()
        
        
    

1 Claim object (4191)
1 Claim object (4190)
1 Claim object (4189)
1 Claim object (4186)
1 Claim object (4180)
1 Claim object (4171)
1 Claim object (4160)
1 Claim object (4112)
1 Claim object (4109)
1 Claim object (4108)
1 Claim object (4107)
1 Claim object (4100)
1 Claim object (4091)
1 Claim object (4085)
1 Claim object (4082)
1 Claim object (4071)
1 Claim object (4068)
1 Claim object (4066)
1 Claim object (4040)
1 Claim object (4038)
1 Claim object (4030)
1 Claim object (4029)
1 Claim object (4028)
1 Claim object (4024)
1 Claim object (4023)
1 Claim object (4021)
1 Claim object (4020)
1 Claim object (4018)
1 Claim object (4013)
1 Claim object (4011)
1 Claim object (4010)
1 Claim object (4009)
1 Claim object (4008)
1 Claim object (3807)
1 Claim object (3800)
1 Claim object (3799)
1 Claim object (3754)
1 Claim object (3753)
1 Claim object (3752)
1 Claim object (3751)


# Get Open Ended

In [11]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from conversations.models import Conversation, Claim

for claim in Claim.objects.all():
    print(claim.id)
    claim.save()
    

    

4215
4214
4213
4212
4211
4210
4209
4208
4207
4206
4205
4204
4203
4202
4201
4200
4199
4198
4197
4196
4195
4194
4193
4192
4191
4190
4189
4188
4187
4186
4185
4184
4183
4182
4181
4180
4179
4178
4177
4176
4175
4174
4173
4172
4171
4170
4169
4168
4167
4166
4165
4164
4163
4162
4161
4160
4159
4158
4157
4156
4155
4154
4153
4152
4151
4150
4149
4148
4147
4146
4145
4144
4143
4142
4141
4140
4139
4138
4137
4136
4135
4134
4133
4132
4131
4130
4129
4128
4127
4126
4125
4124
4123
4122
4121
4120
4119
4118
4117
4116
4115
4114
4113
4112
4111
4110
4109
4108
4107
4106
4105
4104
4103
4102
4101
4100
4099
4098
4097
4096
4095
4094
4093
4092
4091
4090
4089
4088
4087
4086
4085
4084
4083
4082
4081
4080
4079
4078
4077
4076
4075
4074
4073
4072
4071
4070
4069
4068
4067
4066
4065
4064
4063
4062
4061
4060
4059
4058
4057
4056
4055
4054
4053
4052
4051
4050
4049
4048
4047
4046
4045
4044
4043
4042
4041
4040
4039
4038
4037
4036
4035
4034
4033
4032
4031
4030
4029
4028
4027
4026
4025
4024
4023
4022
4021
4020
4019
4018
4017
4016


# Create Conversation survey votes

In [3]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from conversations.models import Conversation, Claim
from surveys.models import ConversationSurvey, ConversationSurveyVote
from login.models import User


MAX_VOTES = 100

all_surveys = ConversationSurvey.objects.all()
all_users = User.objects.all()

for survey in all_surveys:
    
    print(survey.id)
    
    up_votes = random.randint(0, MAX_VOTES)
    down_votes = random.randint(0, MAX_VOTES)
    
    voters = random.sample(list(all_users), k=(up_votes +  down_votes))
    up_voters = voters[:up_votes]
    down_voters = voters[up_votes:]
    
    survey.up_votes = len(up_voters)
    survey.down_votes = len(down_voters)
    survey.save(update_fields=['up_votes', 'down_votes'])
    
    conversation = survey.conversation
    
    conversation.votes += len(up_voters) + len(down_voters)
    conversation.save(update_fields=['votes'])
    
    for voter in up_voters:
        
        vote = ConversationSurveyVote(
            user=voter,
            survey=survey,
            value=1, 
        )
        vote.save()
        
    for voter in down_voters:
        
        vote = ConversationSurveyVote(
            user=voter,
            survey=survey,
            value=-1, 
        )
        vote.save()

3
4
5
2
1


In [6]:
for survey in all_surveys:
    survey.save()

In [17]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from conversations.models import Conversation, Claim
from surveys.models import ConversationSurvey, ConversationSurveyVote
from login.models import User


Conversation.objects.filter(
    Q(open_ended=True) &
    (~Q(summaries__type='ANS')| Q(last_updated__gt=F('summaries__last_updated')) | Q(summaries__isnull=True) )
    #     Q(openended=True) 
#     (Q(summaries__isnull=True) | 
#     (Q(last_updated__gt=F('summaries__last_updated'))))
).distinct()





<QuerySet [<Conversation: Conversation object (19)>]>

In [26]:
conversations = Conversation.objects.filter(
    Q(open_ended=True) &
    (
        ~Q(summaries__type='ANS') |
        Q(last_updated__gt=F('summaries__last_updated')) |
        Q(summaries__isnull=True)
    )
).distinct()
    
    
for conversation in conversations:
    
    claims = conversation.claims.filter(
        parent__isnull=True,
        pvalue__lte=-0.9
    ).distinct().order_by('pvalue')
    
    text_array = [claim.text for claim in claims]
    
    print(text_array)

['Incremental Learning. Learning in small pieces. \\[[1](https://supermemo.guru/wiki/Advantages_of_incremental_reading),[2](http://super-memory.com/help/il.htm),[3](https://supermemo.guru/wiki/Disadvantages_of_incremental_reading)\\]', 'A knowledge an skills should be learned only if it will be soon used for a project.', 'Discussion-based learning.', "Folder for theses mostly relating to skill rather than knowledge. E.x. music, hockey, typing. Don't vote this thesis.", 'Semantic Graphs', '[Multisensory learning](https://en.wikipedia.org/wiki/Multisensory_learning). Sub theses. Learning with more than one sense. Power point presentation with slides including pictures. Educational game.', 'Books.', 'Take notes while using information sources like books and lectures.', '\\(when writing\\)/\\(when choosing a book look for\\) Ask thought provoking question about material before presenting the material.', 'Online Classroom. Teleconference and VR virtual classroom. Not an institution.']


In [5]:
import re

def find_md_links(md):
    """Returns dict of links in markdown:
    'regular': [foo](some.url)
    'footnotes': [foo][3]
    
    [3]: some.url
    """
    # https://stackoverflow.com/a/30738268/2755116

    INLINE_LINK_RE = re.compile(r'\[([^\]]+)\]\(([^)]+)\)')
    FOOTNOTE_LINK_TEXT_RE = re.compile(r'\[([^\]]+)\]\[(\d+)\]')
    FOOTNOTE_LINK_URL_RE = re.compile(r'\[(\d+)\]:\s+(\S+)')

    links = list(INLINE_LINK_RE.findall(md))
    footnote_links = dict(FOOTNOTE_LINK_TEXT_RE.findall(md))
    footnote_urls = dict(FOOTNOTE_LINK_URL_RE.findall(md))

    footnotes_linking = []
        
    for key in footnote_links.keys():
        footnotes_linking.append((footnote_links[key], footnote_urls[footnote_links[key]]))

    return {'regular': links, 'footnotes': footnotes_linking}


def replace_md_links(md):
    """Replace links url to f(url)"""
    
    links = find_md_links(md)
    newmd = md

    for r in links['regular']:
        newmd = newmd.replace('({})'.format(r[1]), '')
        newmd = newmd.replace('[{}]'.format(r[0]), r[0])

    for r in links['footnotes']:
        newmd = newmd.replace(r[1], f(r[1]))
    
    return newmd



text = 'test [etts](url)'

replace_md_links(text)

'test etts'

In [38]:
conversation.claims.values_list('author', flat=True).distinct() | conversation.claims.values_list('votes__user', flat=True).distinct()

TypeError: Merging 'QuerySet' classes must involve the same values in each case.

In [30]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from administration.models import Customer
from conversations.models import Conversation
from django.db.models import Q, F

conversations = Conversation.objects.filter(
    Q(summaries__isnull=True) | Q(last_updated__gt=F('summaries__last_updated'))
).distinct()

conversations

<QuerySet [<Conversation: Conversation object (35)>, <Conversation: Conversation object (24)>, <Conversation: Conversation object (1)>]>

In [7]:
import json
import random
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from administration.models import Customer
from conversations.models import Conversation
from login.models import User
from django.db.models import Q, F


user = User.objects.get(id=1)
user.username

'w9PFe8wKWsZibGraFJfvU8cVaPl2'

In [28]:
import names
import firebase_admin
from firebase_admin import auth, credentials

cred = credentials.Certificate("./../firebase-config-staging.json")
app = firebase_admin.initialize_app(name="motivee", credential=cred)
# app = firebase_admin.get_app(name="motivee")
client = auth.Client(app)

In [32]:
user = client.get_user_by_email('dstratford84@gmail.com')
user.

False

In [15]:
import json

colorado = {
    'Division': [
        "Dean's Office",
        'Arts & Humanities',
        'Natural Sciences',
        'Social Sciences'
    ],
    'Job title': [
        'Student employees', 
        'Staff', 
        'Teaching faculty',
        'Tenure track faculty', 
    ]
}

json.dumps(colorado)

'{"Division": ["Dean\'s Office", "Arts & Humanities", "Natural Sciences", "Social Sciences"], "Job title": ["Student employees", "Staff", "Teaching faculty", "Tenure track faculty"]}'